In [11]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import __credential__

# Setup Driver for connection
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/spark/jars/postgresql-42.2.2.jar pyspark-shell'
environ['PYSPARK_PYTHON']='/home/ubuntu/anaconda3/bin/python'
environ['PYSPARK_DRIVER_PYTHON']='/home/ubuntu/anaconda3/bin/jupyter'

In [12]:
spark = SparkSession \
    .builder \
    .master(__credential__.spark_host) \
    .appName("txt_reader") \
    .getOrCreate()

### Read RNA-seq files of FPKM-UQ format

In [8]:
# in tabular format of: [gene_id \t expression_value]
fpkm = spark.read.format("csv")\
    .option("delimiter","\t").option("quote","")\
    .option("header", "false")\
    .option("inferSchema", "true")\
    .load(__credential__.modeule_txt_reader_testfile)

# Didn't find a way to assign schema after taking in the data
#fpkm.setSchema(StructType([ \
#    StructField("A", StringType(), True), StructField("B", FloatType(), True)]))

# Assign schema 
fpkm_rdd = fpkm.rdd.map(lambda x:Row(gene_id=x[0], expr_val=x[1]))
fpkm_rdd.take(5)

[Row(expr_val=0.0, gene_id='ENSG00000242268.2'),
 Row(expr_val=0.0, gene_id='ENSG00000270112.3'),
 Row(expr_val=171012.179191, gene_id='ENSG00000167578.15'),
 Row(expr_val=0.0, gene_id='ENSG00000273842.1'),
 Row(expr_val=48643.8412233, gene_id='ENSG00000078237.5')]

### Read reference genome pre-stored in PostgreSQL database on Amazon AWS

In [9]:
ref_genome = spark.read \
    .format('jdbc') \
    .option('url', 'jdbc:postgresql://%s' % __credential__.jdbc_accessible_host) \
    .option('dbtable', __credential__.dbtable) \
    .option('user', __credential__.user) \
    .option('password', __credential__.password) \
    .load()
ref_genome_rdd = ref_genome.rdd

### Convert gene_id to gene_name via joining 2 tables

In [10]:
# Filter all genes with expression level = 0
# Truncate the gene id from for eg. `ENSG00000007080.9` to `ENSG00000007080`
fpkm_trunc = fpkm_rdd \
                .filter(lambda x: x.expr_val > 0) \
                .map(lambda x : Row( \
                        gene_id=x.gene_id.split('.')[0], \
                        expr_val=x.expr_val)) \
                .toDF()  # .count() = 30361
            
# Truncate the gene id from for eg. `ENSG00000007080.9` to `ENSG00000007080`
ref_genome_trunc = ref_genome_rdd \
                .map(lambda x : Row( \
                        gene_id=x.id.split('.')[0], gene_name=x.name)) \
                .toDF()
        
# Join to get table: gene_id | gene_name | expr_val
match = fpkm_trunc.join(ref_genome_trunc, 'gene_id')
match.count()  # 29907 
match.show(10)

+---------------+-------------+---------+
|        gene_id|     expr_val|gene_name|
+---------------+-------------+---------+
|ENSG00000059588|115837.535272|   TARBP1|
|ENSG00000070182|234.534025063|     SPTB|
|ENSG00000070366|50443.7932222|     SMG6|
|ENSG00000072071|186464.397019|   ADGRL1|
|ENSG00000073536|93506.6221732|     NLE1|
|ENSG00000075290|182.267288417|    WNT8B|
|ENSG00000083454|459.365767466|    P2RX5|
|ENSG00000083782|1279.60724588|     EPYC|
|ENSG00000086200|86026.2923919|    IPO11|
|ENSG00000087087|1011790.34872|     SRRT|
+---------------+-------------+---------+
only showing top 10 rows



### Convert gene_id to gene_name via joining 2 tables - check

In [ ]:
# Check step: 
# In test case b9e7b7b5-54e8-4459-8a21-ea3472b013d7.FPKM-UQ.txt,
# There are in total 60483 fpkm counts, and 64561 gene entries in ref genome
# However, only 37670 entries after inner joinning on gene_id.
# Thus, I'm using left outer join to check which gene entries are missing
left = fpkm.join(ref_genome, fpkm[0] == ref_genome.id, "left_outer")
left.count()  # 60483
missing = left.rdd.filter(lambda x : x.id == None).map(lambda x : x[0])
missing.count()  # 22813

# The gene ids in ensembl database (ref genome) and RNA-seq data contains decimal,
# such as `ENSG00000007080.9`
# However, all ``ENSG00000007080.*` might refer to the same gene
# So the decimal is removed for mapping purpose
fpkm_trunc = fpkm_rdd.map(lambda x : Row( \
            gene_id=x.gene_id.split('.')[0], expr_val=x.expr_val)).toDF()
ref_trunc = ref_genome_rdd.map(lambda x : Row( \
            id=x.id.split('.')[0], name=x.name)).toDF()
left_trunc = fpkm_trunc.join(ref_trunc, fpkm_trunc.gene_id == ref_trunc.id, "left_outer")
missing_trunc = left_trunc.rdd.filter(lambda x : x.id == None).map(lambda x : x.gene_id)
missing_trunc.count()  # 3612
# The missing ones, such as `ENSG00000221077`, is not found in the downloaded ref genome